In [1]:
import os
import random
import sys
import math
import pickle
import numpy as np
import pandas as pd
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset ,ConcatDataset
sys.path.append(r'D:\WorkPath\Models\SwinGAN')
from MyDataSet import MultiMaskTimeSeriesDataset

def cal_MAPE(y_true,y_pred,mask):
    y_true=y_true[mask]
    y_pred=y_pred[mask]
    return np.mean(np.abs(y_true-y_pred)/(y_true+1e-2))


In [2]:
def Historic_Average(data : MultiMaskTimeSeriesDataset, iter ,history_len = 7):


    test = np.random.randint(7*data.num_masks, len(data)-1 , iter)
    total_MSE = []
    total_MAE = []
    for i in range(iter):
        index = test[i]

        history_data = data.get_historical_data(index, history_len)
        HA=np.mean(history_data, axis=0)
        true_data = np.array(data[index][0])
        mask = np.array(data[index][1])
        # print(mask)
        
        # 计算MSE
        HA = HA * (1 - mask)
        true_data = true_data * (1 - mask)
        total_MSE.append(np.sum((HA - true_data)**2)/np.sum(1-mask))
        total_MAE.append(cal_MAPE(true_data,HA,(1 - mask).astype(bool)   ))
    return total_MSE,total_MAE

In [ ]:
project_path = r'D:\WorkPath\Models\ImputeFormer'
test_path = os.path.join(project_path , r'Data\source_test_PEMS04') 
test_files = os.listdir(test_path)
test_files = [os.path.join(test_path, file) for file in test_files]

test_record = {'data_name':[],'MSE_test_loss':[] , 'MAPE_test_loss':[]}

for file_path in test_files:
    with open(file_path, 'rb') as f:
        test_data = pickle.load(f)

    total_MSE,total_MAE=Historic_Average(test_data,300 , 3)
    test_record['data_name'].append(file_path)
    test_record['MSE_test_loss'].append(np.mean(total_MSE))
    test_record['MAPE_test_loss'].append(np.mean(total_MAE))
test_record = pd.DataFrame(test_record)
test_record['route']=test_record['data_name'].apply(lambda x :x.split('_')[5])
test_record['start']=test_record['data_name'].apply(lambda x :x.split('_')[-3])
test_record['miss_rate']=test_record['data_name'].apply(lambda x :x.split('_')[-2])
test_record['type']=test_record['data_name'].apply(lambda x :x.split('_')[-1][:-4])
test_record=test_record[['route','start','miss_rate','type','MSE_test_loss','MAPE_test_loss']]
test_record=test_record.sort_values(['route','type','start'])
test_record